<a href="https://colab.research.google.com/github/jumbokh/ML-Class/blob/main/notebooks/4_3_SVM_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 作業

先用以下程式碼載入資料


df = pd.read_csv('titanic_train.csv')  
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)  
X = df.drop('Survived', axis=1)  
y = df['Survived']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
                                                   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['DFKai-sb'] 
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('titanic_train.csv')
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X = df.drop('Survived', axis=1)
y = df['Survived']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.33, random_state=42)
X.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

1. 請做出三個資料管道器
    * log_pl包含遺漏值(平均值), FunctionTransformer(np.log1p), 標準差
    * num_pl包含遺漏值(平均值), 標準差
    * cat_pl包含遺漏值(用常數'missing'), 獨熱編碼
    
其中['Age','Fare']去log_pl，['SibSp','Parch','Pclass']去num_pl，['Sex','Embarked']去cat_pl，請問最後的正確率是多少？

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

log_pl = make_pipeline(
    SimpleImputer(strategy='mean'),
    FunctionTransformer(np.log1p),
    StandardScaler()
)
num_pl = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)
cat_pl = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder()
)

data_pl = ColumnTransformer([
    ('log_pl', log_pl, ['Age','Fare']),
    ('num_pl', num_pl, ['SibSp','Parch','Pclass']),
    ('cat_pl', cat_pl, ['Sex','Embarked'])
])

model_pl = make_pipeline(data_pl, SVC())
model_pl.fit(X_train, y_train)
y_pred = model_pl.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print('整體正確率:',accuracy_score(y_test, y_pred).round(2))

[[159  16]
 [ 36  84]]
整體正確率: 0.82


2. 承上，將支持向量機改為K最近鄰法來做預測  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_pl_knn = make_pipeline(data_pl, KNeighborsClassifier())
model_pl_knn.fit(X_train, y_train)
y_pred = model_pl_knn.predict(X_test)
print('正確率：', accuracy_score(y_test, y_pred).round(2))
print('混亂矩陣')
print(confusion_matrix(y_test, y_pred))

正確率： 0.81
混亂矩陣
[[155  20]
 [ 36  84]]
